In [0]:
%sql
-- =========================================================
-- ETAPA 0: Pruebas de escritorio
-- Objetivo:
--   Definir pruebas en carga de datos 
-- =========================================================
--CREATE OR REPLACE TEMP VIEW silver_base AS
--SELECT *
--FROM tfm_silver.vehiculos
--WHERE source_year = 2025;      


In [0]:
%sql
-- =========================================================
-- ETAPA 1: Construcción del lote para GOLD
-- Objetivo:
--   1️ Enriquecer los datos de SILVER con catálogos (códigos → nombres)
--   2️ Filtrar solo vehículos nuevos (según reglas)
--   3️ Quitar duplicados (un registro por vehículo)
-- =========================================================

CREATE OR REPLACE TEMP VIEW vista_vehiculos_candidatos AS
WITH datos_enriquecidos AS (
  -- =========================================================
  -- Paso 1: Unir la tabla principal con los catálogos para
  -- reemplazar los ID numéricos por descripciones legibles.
  -- =========================================================
  SELECT
    s.source_year            AS ANIO_ORIGEN,          -- Año de origen del dataset
    s.CODIGO_VEHICULO,                                -- Identificador único del vehículo
    tt.TIPO_TRANSACCION,                              -- Tipo de transacción 
    m.MARCA,                                          -- Marca del vehículo
    s.MODELO,                                         -- Modelo del vehículo
    p.PAIS,                                           -- País de origen
    s.ANIO_MODELO,                                   -- Año del modelo del vehículo
    c.CLASE,                                          -- Clase (automóvil, motocicleta, etc.)
    sc.SUBCLASE,                                      -- Subclase del vehículo
    CASE
      -- Regla: si la clase es "motocicleta", el tipo de vehículo será "EXTRALIVIANO"
      WHEN UPPER(TRIM(c.CLASE)) = 'MOTOCICLETA' THEN 'EXTRALIVIANO'
      ELSE tv.TIPO_VEHICULO
    END AS TIPO_VEHICULO,
    s.AVALUO,                                         -- Valor comercial
    s.FECHA_PROCESO,                                  -- Fecha en que se procesó el dato
    ts.TIPO_SERVICIO,                                 -- Tipo de servicio (particular, comercial, etc.)
    s.CILINDRAJE,                                     -- Cilindraje del motor
    tc.TIPO_COMBUSTIBLE,                              -- Tipo de combustible
    s.FECHA_COMPRA,                                   -- Fecha en que se compró el vehículo
    s.CANTON        AS ID_CANTON,                     -- Identificador del cantón
    s.COLOR1,                                         -- Color principal
    tcomp.TIPO_COMPRADOR,                             -- Tipo de comprador (natural, jurídica, etc.)
    s.ingest_ts AS FECHA_INGESTA                      -- Fecha en que el registro entró a SILVER
  FROM tfm_silver.vehiculos s --silver_base s
  -- Joins con los catálogos de SILVER
  LEFT JOIN tfm_silver_ctg.cat_tipo_transaccion tt ON s.tipo_transaccion_id = tt.tipo_transaccion_id
  LEFT JOIN tfm_silver_ctg.cat_marca            m  ON s.marca_id           = m.marca_id
  LEFT JOIN tfm_silver_ctg.cat_pais             p  ON s.pais_id            = p.pais_id
  LEFT JOIN tfm_silver_ctg.cat_clase            c  ON s.clase_id           = c.clase_id
  LEFT JOIN tfm_silver_ctg.cat_subclase         sc ON s.subclase_id        = sc.subclase_id
  LEFT JOIN tfm_silver_ctg.cat_tipo_vehiculo    tv ON s.tipo_vehiculo_id   = tv.tipo_vehiculo_id
  LEFT JOIN tfm_silver_ctg.cat_tipo_servicio    ts ON s.tipo_servicio_id   = ts.tipo_servicio_id
  LEFT JOIN tfm_silver_ctg.cat_tipo_combustible tc ON s.tipo_combustible_id= tc.tipo_combustible_id
  LEFT JOIN tfm_silver_ctg.cat_tipo_comprador   tcomp ON s.tipo_comprador_id = tcomp.tipo_comprador_id
),

vehiculos_nuevos AS (
  -- =========================================================
  -- Paso 2: Filtrar solo los vehículos NUEVOS
  -- Aplicando las reglas de negocio:
  --   - Eliminar compras antiguas (1900–2016)
  --   - Aceptar modelos recientes (mayor o igual al año del dataset - 1)
  -- =========================================================
  SELECT *
  FROM datos_enriquecidos
  WHERE
    YEAR(FECHA_COMPRA) >= 2016  -- Quita los usados antiguos
    AND (ANIO_MODELO >= (ANIO_ORIGEN - 1))         -- Asegura que el modelo sea reciente
),

vehiculos_deduplicados AS (
  -- =========================================================
  -- Paso 3: Eliminar duplicados (mismo código)
  -- Criterios de prioridad:
  --   1️ FECHA_COMPRA más antigua
  --   2️ Si empata, FECHA_PROCESO más antigua
  -- =========================================================
  SELECT
    v.*,
    ROW_NUMBER() OVER (
      PARTITION BY CODIGO_VEHICULO
      ORDER BY FECHA_COMPRA ASC NULLS LAST, FECHA_PROCESO ASC NULLS LAST
    ) AS numero_fila
  FROM vehiculos_nuevos v
)

-- =========================================================
-- Paso 4: Resultado final del lote candidato
-- Solo conserva la primera fila por cada vehículo (numero_fila = 1)
-- =========================================================
SELECT
  ANIO_ORIGEN,
  CODIGO_VEHICULO,
  TIPO_TRANSACCION,
  MARCA,
  MODELO,
  PAIS,
  ANIO_MODELO,
  CLASE,
  SUBCLASE,
  TIPO_VEHICULO,
  AVALUO,
  FECHA_PROCESO,
  TIPO_SERVICIO,
  CILINDRAJE,
  TIPO_COMBUSTIBLE,
  FECHA_COMPRA,
  ID_CANTON,
  COLOR1,
  TIPO_COMPRADOR,
  FECHA_INGESTA
FROM vehiculos_deduplicados
WHERE numero_fila = 1;


In [0]:
%sql
-- =========================================================
-- ETAPA 2: Inserción en la tabla GOLD (vehículos)
-- Objetivo:
--   Insertar SOLAMENTE los vehículos que no existen todavía
--   en la tabla maestra GOLD.
-- =========================================================

INSERT INTO tfm_gold.vehiculos
SELECT
  c.ANIO_ORIGEN,         -- Año del dataset de origen (para particionar)
  c.CODIGO_VEHICULO,     -- Identificador único del vehículo
  c.TIPO_TRANSACCION,    -- Tipo de transacción (compra, venta, etc.)
  c.MARCA,               -- Marca del vehículo
  c.MODELO,              -- Modelo del vehículo
  c.PAIS,                -- País de origen o fabricación
  c.ANIO_MODELO,         -- Año del modelo
  c.CLASE,               -- Clase del vehículo (auto, moto, camión, etc.)
  c.SUBCLASE,            -- Subclase más específica
  c.TIPO_VEHICULO,       -- Tipo general del vehículo
  c.AVALUO,              -- Valor comercial o fiscal
  c.FECHA_PROCESO,       -- Fecha en la que el registro fue procesado
  c.TIPO_SERVICIO,       -- Tipo de servicio (particular, comercial, etc.)
  c.CILINDRAJE,          -- Cilindraje del motor
  c.TIPO_COMBUSTIBLE,    -- Tipo de combustible
  c.FECHA_COMPRA,        -- Fecha en que fue comprado
  c.ID_CANTON,           -- Código del cantón
  c.COLOR1,              -- Color principal
  c.TIPO_COMPRADOR,      -- Tipo de comprador (natural, jurídica, etc.)
  CURRENT_TIMESTAMP() AS FECHA_INSERCION  -- Fecha actual de inserción en GOLD
FROM vista_vehiculos_candidatos c
-- =========================================================
-- Solo insertar si el vehículo NO existe todavía en GOLD.
-- =========================================================
LEFT ANTI JOIN tfm_gold.vehiculos g
  ON g.CODIGO_VEHICULO = c.CODIGO_VEHICULO;
-- LEFT ANTI JOIN: deja solo los vehículos que aún no existen en GOLD


In [0]:
%sql
SELECT COUNT(*) FROM tfm_gold.vehiculos